Reference: ("Prediction") https://github.com/google/earthengine-community/blob/master/guides/linked/UNET_regression_demo.ipynb

In [30]:
import ee
import folium
import geopandas as gpd
import tensorflow as tf
import numpy as np

In [2]:
ee.Initialize()

# 1. Convert the sample image to a TensorFlow record

In [257]:
# Read in the image from EE
sample_image = ee.Image("projects/esg-satelite/assets/mars/modelling/model_input/test1")

In [258]:
# Create "stack"
image_stack = ee.Image.cat(
    [
        sample_image
    ]
)

In [259]:
image_stack = image_stack.float()

In [260]:
task = ee.batch.Export.image.toCloudStorage(
    image=image_stack,
    description="TensorFlow record of sample area of THEMIS Tempe Terra.",
    bucket="esg-satelite-data-warehouse",
    fileNamePrefix=f"mars/predictions/features/themis_sample_1/themis_sample_1",
    fileFormat = "TFRecord",
    region = ee.Feature(sample_image.geometry()).geometry(),
    scale = 500,
    formatOptions = {
        "patchDimensions": [256,256],
        "maxFileSize": 104857600,
        "compressed": True,
    }
)

In [261]:
task.start()

In [263]:
task.status()

{'state': 'COMPLETED',
 'description': 'TensorFlow record of sample area of THEMIS Tempe Terra.',
 'creation_timestamp_ms': 1696867150652,
 'update_timestamp_ms': 1696867170117,
 'start_timestamp_ms': 1696867160634,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.14684510231018066,
 'id': '6AWUB4BOZIRNNTZMT2CIFW3G',
 'name': 'projects/earthengine-legacy/operations/6AWUB4BOZIRNNTZMT2CIFW3G'}

## 1.1. Plotting on Folium

In [51]:
# Creating add_ee_layer function
# This allows us to visualise EE objects on a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee_image_object.getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [18]:
Map = folium.Map() 

Map.add_ee_layer(
    sample_image,
    {},
    "Tempe Terra"
)

Map.add_ee_layer(
    ee.FeatureCollection(sample_image.geometry()),
    {"color": "red"},
    "Geometry"
)

_ = folium.LayerControl().add_to(Map)


In [19]:
Map

# 2. Read in image

In [20]:
imageFilesList = ["gs://esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1/themis_sample_1.tfrecord.gz"]

In [21]:
image_dataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')

In [22]:
BANDS = ["b1"]

In [23]:
imageColumns = [
    tf.io.FixedLenFeature(shape=[256,256], dtype=tf.float32) 
      for k in BANDS
  ]

imageFeaturesDict = dict(zip(BANDS, imageColumns))

def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

In [24]:

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

In [25]:
image_dataset

<_ParallelMapDataset element_spec={'b1': TensorSpec(shape=(256, 256), dtype=tf.float32, name=None)}>

In [26]:
def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

In [27]:
image_dataset = image_dataset.map(toTupleImage).batch(1)

In [28]:
image_dataset

<_BatchDataset element_spec=TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None)>

In [31]:
np.expand_dims(np.concatenate([x for x in image_dataset]), axis = 0).shape

(1, 1, 256, 256, 1)

##### Reference: https://github.com/google/earthengine-community/blob/master/guides/linked/TF_demo1_keras.ipynb "Read the JSON mixer file"

In [345]:
import json
import numpy as np

In [346]:
json_file = "gs://esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1/themis_sample_1.json"

In [347]:
image_files = [
    "gs://esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1/themis_sample_1.tfrecord.gz"
]

In [348]:
json_text = !gsutil cat {json_file}

In [349]:
mixer = json.loads(json_text.nlstr)

In [350]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

In [351]:
patch_dimensions_flat

[65536, 1]

In [352]:
BANDS = ["b1"]


In [353]:
# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

In [354]:
# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

In [355]:
image_features_dict

{'b1': FixedLenFeature(shape=[65536, 1], dtype=tf.float32, default_value=None)}

In [356]:
# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files, compression_type= "GZIP")

In [357]:
image_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [333]:
# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

In [334]:
image_dataset

<ParallelMapDataset element_spec={'b1': TensorSpec(shape=(65536, 1), dtype=tf.float32, name=None)}>

In [335]:
# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
) 

In [336]:
image_dataset

<FlatMapDataset element_spec={'b1': TensorSpec(shape=(1,), dtype=tf.float32, name=None)}>

In [337]:
# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

In [338]:
image_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None),)>

In [339]:
np.expand_dims(np.concatenate([x for x in image_dataset]), axis = 0).shape

(1, 1, 65536, 1, 1)

# 3. Get model

In [16]:
import tensorflow as tf

In [17]:
model = tf.keras.models.load_model("gs://esg-satelite-data-warehouse/mars/models/sample_model_1")

In [32]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=1, verbose=1)

2023-10-09 16:29:37.772659: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:435] Loaded runtime CuDNN library: 8.2.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-10-09 16:29:37.774535: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_impl.h:1592 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Graph execution error:

Detected at node model_1/conv2d_23/Conv2D defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 516, in dispa